In [1]:
# To use math functions
import numpy as np
np.set_printoptions(suppress=True)

# To read csv files
import pandas as pd

#To plot data
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

#select scenario
num_scenario=1

# read csv
df_lidar = pd.read_csv('scenario_%d/scan.csv'%num_scenario)
df_encoder=pd.read_csv('scenario_%d/sensor_state.csv'%num_scenario)

# change index
df_lidar= df_lidar.rename(columns={"%time": "time"})
df_lidar['time'] = (df_lidar['time'] - df_lidar['time'].tolist()[0])/1000000000
df_lidar.set_index('time',inplace=True)
# strip column names
df_lidar.columns = [name[6:] for name in df_lidar]

df_encoder= df_encoder.rename(columns={"%time": "time"})
df_encoder['time'] = (df_encoder['time'] - df_encoder['time'].tolist()[0])/1000000000
df_encoder.set_index('time',inplace=True)
# strip column names
df_encoder.columns = [name[6:] for name in df_encoder]

###################TP3
left=np.array(df_encoder.left_encoder)
right=np.array(df_encoder.right_encoder)
r=33/1000
L=80/1000

diff_phi_g =left[1:]-left[:-1]
diff_phi_d =right[1:]-right[:-1]
diff_temps=df_encoder.index[1:]

phi_prime_g = diff_phi_g *2* np.pi/4096/diff_temps
phi_prime_d = diff_phi_d *2 *np.pi/4096/diff_temps

v=r*(phi_prime_g+phi_prime_d)/2
omega=r*(phi_prime_d-phi_prime_g)/(2*L)

theta=np.zeros(len(df_encoder.index))
x=np.zeros(len(df_encoder.index))
y=np.zeros(len(df_encoder.index))
for k in range(1,len(df_encoder.index)):
    t= diff_temps[k-1]
    theta[k]=theta[k-1]+ omega[k-1]*t
    x[k]=x[k-1]+ v[k-1]*np.cos(theta[k])*t
    y[k]=y[k-1]+ v[k-1]*np.sin(theta[k])*t

# chercher une correspondance entre le temps de df_lidar et df_encoder  
# car la precision de temps est differente
match_index= np.zeros(len(df_lidar)).astype(int)
for i in range(len(df_lidar)):
    min= np.inf
    for j in range(len(df_encoder)):
        if(abs(df_lidar.index[i]-df_encoder.index[j])<min):
            min=abs(df_lidar.index[i]-df_encoder.index[j])
            match_index[i]=j
            
theta_match = theta[match_index]
x_match = x[match_index]
y_match = y[match_index]

In [2]:
%matplotlib auto

plt.figure(figsize=(4,4))
fig = plt.gcf()
ax= plt.gca()
plt.ion()
list_angle_ecart=[93,63.5,83,94]
list_x_ecart=[0.75,0.3,0.4,0.4]
list_y_ecart=[0.3,0.2,0.3,0.5]
angle_ecart=list_angle_ecart[num_scenario-1]
x_ecart=list_x_ecart[num_scenario-1]
y_ecart=list_y_ecart[num_scenario-1]

for time, row in df_lidar.iterrows():

    n = 360  # <-- number of points
    ranges = np.array(row[10:10+n])
    intensities = np.array(row[10+n:10+2*n])
    idx=np.argwhere(df_lidar.index==time).flatten()[0]
    angle_sol= angle_ecart*np.pi/180
    angle=theta_match[idx]+angle_sol

    scan=[]
    scan.append(ranges*np.array([np.cos(i*np.pi/180+angle) for i in range(360)]))   
    scan.append(ranges*np.array([np.sin(i*np.pi/180+angle) for i in range(360)]))
    
    #filtre
    while(abs(scan[0]).min()+abs(scan[1]).min()<0.0001):
        intensities=np.delete(intensities,abs(scan[1]).argmin())
        scan[0]=np.delete(scan[0],abs(scan[1]).argmin())
        scan[1]=np.delete(scan[1],abs(scan[1]).argmin())

    matrice_rot=np.array(
        [[np.cos(angle_sol),-np.sin(angle_sol)],
         [np.sin(angle_sol),np.cos(angle_sol)]])    
    
    x_rot,y_rot=matrice_rot@np.array([x_match,y_match])
    x_rot+=x_ecart
    y_rot+=y_ecart
    
    plt.plot(0,0,"ko",label="Original")
    plt.scatter(scan[0]+x_rot[idx], scan[1]+y_rot[idx], c=intensities)
    plt.plot(x_rot[idx],y_rot[idx],"ro",
             label='robot:(%.3f m, %.3f m)'%(x_rot[idx],y_rot[idx])
            )
    arrow_l=0.2
    plt.arrow(x_rot[idx],
              y_rot[idx],
              arrow_l*np.cos(angle),
              arrow_l*np.sin(angle),
              color="r",width=0.005, 
             head_width=0.05,
              head_length=0.07, 
              overhang=-0.2
             )
    
    plt.autoscale(False)
    plt.legend(title="Scenario: %d\ntime: %.3f s \nAngle: %.3f°"%(num_scenario,time,angle*180/np.pi))
    
    my_x_ticks = np.arange(-0.5, 3.2, 0.4)
    my_y_ticks = np.arange(-0.5, 3.2, 0.4)
    plt.xticks(my_x_ticks)
    plt.yticks(my_y_ticks)
    plt.draw()
    plt.pause(0.01)
    plt.clf()

Using matplotlib backend: Qt5Agg
